In [9]:
import re
import glob
import os
import pandas as pd
import csv
import unicodedata

## Juntar os arquivos para ter um único arquivo do ano de referência

O ano de 2024 tem apenas os dois primeiros quadrimestres.

In [6]:
# Caminho dos arquivos
caminho_arquivos_csv = "../data/terceirizados/*.csv"
caminho_arquivos_xlsx = "../data/terceirizados/*.xlsx"

# Encontrar todos os arquivos CSV e XLSX
arquivos = glob.glob(caminho_arquivos_csv) + glob.glob(caminho_arquivos_xlsx)

# Criar um dicionário para armazenar os DataFrames por ano
dfs_por_ano = {}
arquivos_ignorados = []  # Lista de arquivos que não puderam ser lidos

In [7]:
# Função para tentar diferentes encodings ao ler CSVs
def ler_csv_com_encoding(arquivo):
    encodings = ['utf-8', 'ISO-8859-1', 'windows-1252']
    for enc in encodings:
        try:
            df = pd.read_csv(arquivo, sep=';', dtype=str, encoding=enc)
            print(f"✅ Arquivo lido com sucesso: {arquivo} (Encoding: {enc})")
            return df
        except UnicodeDecodeError:
            print(f"⚠️ Erro com encoding {enc} para {arquivo}, tentando outro...")
    print(f"❌ Arquivo ignorado: {arquivo} (Não foi possível abrir com nenhum encoding)")
    arquivos_ignorados.append(arquivo)
    return None  # Retorna None se nenhum encoding funcionar

In [8]:
# Processar cada arquivo corretamente
for arquivo in arquivos:
    if arquivo.endswith(".csv"):
        df = ler_csv_com_encoding(arquivo)
    elif arquivo.endswith(".xlsx"):
        df = pd.read_excel(arquivo, dtype=str)  # Ler XLSX normalmente
        print(f"✅ Arquivo lido com sucesso: {arquivo} (Formato: XLSX)")
    else:
        continue  # Agora não processamos diretórios ou arquivos inválidos

    if df is None:
        continue  # Pular arquivos que não puderam ser lidos

    # Extrair o ano do nome do arquivo
    match = re.search(r'(\d{4})', arquivo)
    if match:
        ano = match.group(1)
    else:
        print(f"⚠️ Ano não identificado no arquivo: {arquivo}")
        arquivos_ignorados.append(arquivo)
        continue  # Ignorar arquivos sem ano no nome

    # Adicionar ao dicionário
    if ano in dfs_por_ano:
        dfs_por_ano[ano].append(df)
    else:
        dfs_por_ano[ano] = [df]

✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_202101.csv (Encoding: utf-8)
✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_202301.csv (Encoding: utf-8)
✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_202305.csv (Encoding: utf-8)
✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_202105.csv (Encoding: utf-8)
⚠️ Erro com encoding utf-8 para ../data/terceirizados/terceirizados_202201.csv, tentando outro...
✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_202201.csv (Encoding: ISO-8859-1)
✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_202001.csv (Encoding: utf-8)
✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_201905.csv (Encoding: utf-8)
✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_201901.csv (Encoding: utf-8)
✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_202005.csv (Encoding: utf-8)
⚠️ Erro com encoding utf-8 para ../data/terceirizados/tercei

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x104a81940>>
Traceback (most recent call last):
  File "/Users/rislamiranda/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


✅ Arquivo lido com sucesso: ../data/terceirizados/terceirizados_202405.xlsx (Formato: XLSX)


In [ ]:
# Criar um CSV consolidado para cada ano
caminho_saida = "../data/terceirizados/arquivos_tratados/"

for ano, lista_dfs in dfs_por_ano.items():
    df_ano = pd.concat(lista_dfs, ignore_index=True)  # Junta todos os quadrimestres do ano
    df_ano = df_ano.drop_duplicates()  # Remove linhas idênticas

    # Salvar CSV consolidado no diretório correto
    df_ano.to_csv(f"{caminho_saida}contratos_consolidados_{ano}.csv", index=False, encoding="utf-8")
    print(f"✅ Dados do ano {ano} salvos em '{caminho_saida}contratos_consolidados_{ano}.csv'.")

In [6]:
# Exibir relatório de arquivos ignorados
if arquivos_ignorados:
    print("\n❌ Arquivos ignorados (não puderam ser lidos):")
    for arquivo in arquivos_ignorados:
        print(f"  - {arquivo}")
else:
    print("\n✅ Todos os arquivos foram lidos corretamente!")


✅ Todos os arquivos foram lidos corretamente!


## Padronizar os números de contrato para ter apenas números

In [11]:
df_2019 = pd.read_csv("../data/terceirizados/arquivos_tratados/contratos_consolidados_2019.csv", sep =",", low_memory=False)
df_2019.head()

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
0,7971733,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,01/2018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.906.681-**,LUIS CARLOS PONTES DA SILVA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
1,7971734,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,01/2018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.394.463-**,FRANCISCO JOSE SILVA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
2,7971735,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,01/2018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.243.351-**,MICHEL CHRISTIAN CORREA DE BARROS,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
3,7971736,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,01/2018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.094.541-**,WENDEL BERGUE DE SOUSA MIRANDA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
4,7971737,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,01/2018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.355.123-**,RAFAEL TEIXEIRA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
5,7971738,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,01/2018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.429.743-**,SERGIO CASTRO SILVA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
6,7971739,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,01/2018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.351.113-**,IZAU LIMA DOS SANTOS,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
7,7971740,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,01/2018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.861.401-**,ALESSANDRO CAMPOS DOS SANTOS,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
8,7971741,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,02/2017,09284699000133,AVAL EMPRESA DE SEGURANCA LTDA- EPP.,***.747.661-**,EDIGAR LEANDRO PEREIRA SANTOS,...,BIBLIOTECA DEMONSTRATIVA DE BRASILIA,2616.300048828125,5642.89013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
9,7971742,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,02/2017,09284699000133,AVAL EMPRESA DE SEGURANCA LTDA- EPP.,***.646.221-**,FABIO RODRIGUES MOREIRA,...,BIBLIOTECA DEMONSTRATIVA DE BRASILIA,2983.110107421875,6319.240234375,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000


In [8]:
df_2019.shape

(276554, 23)

In [12]:
df_2020 = pd.read_csv("../data/terceirizados/arquivos_tratados/contratos_consolidados_2020.csv", sep =",", low_memory=False)
df_2020.head()

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
0,7801176,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,623/2017,1863518000111,TERRA VIGILANCIA E SEGURANCA EIRELI,***.715.353-**,ARNALDO DA SILVA MACHADO,...,SUPERINTENDENCIA REGIONAL DNIT GO/DF,2085.209961,4659.609863,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252
1,7801177,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,623/2017,1863518000111,TERRA VIGILANCIA E SEGURANCA EIRELI,***.897.631-**,RODRIGO SANTIAGO MANTANA,...,SUPERINTENDENCIA REGIONAL DNIT GO/DF,2304.479980,4942.879883,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252
2,7801178,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,757/2019,8744139000151,G&E SERVICOS TERCEIRIZADOS LTDA,***.129.471-**,SIMONY CASTRO CORREIA SANTANA,...,UNIDADE LOCAL DE RIO VERDE,1962.699951,4231.959961,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252
3,7801179,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,757/2019,8744139000151,G&E SERVICOS TERCEIRIZADOS LTDA,***.130.461-**,MANOELA PEREIRA GARCIA OLIVEIRA,...,UNIDADE LOCAL DE BRASILIA,2145.000000,5403.729980,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252
4,7801180,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,757/2019,8744139000151,G&E SERVICOS TERCEIRIZADOS LTDA,***.130.461-**,MARCIA RODRIGUES DA SILVA,...,SUPERINTENDENCIA REGIONAL DNIT GO/DF,1962.699951,4343.790039,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252
5,7801181,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,757/2019,8744139000151,G&E SERVICOS TERCEIRIZADOS LTDA,***.748.521-**,RENATA SUZANA DE ANDRADE MELO,...,UNIDADE LOCAL DE URUACU,1962.699951,4173.040039,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252
6,7801182,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,757/2019,8744139000151,G&E SERVICOS TERCEIRIZADOS LTDA,***.919.541-**,DIOGO LIMA FONSECA,...,SUPERINTENDENCIA REGIONAL DNIT GO/DF,2137.649902,4668.240234,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252
7,7801183,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,757/2019,8744139000151,G&E SERVICOS TERCEIRIZADOS LTDA,***.582.581-**,GRACIANE ALVES SILVA,...,SUPERINTENDENCIA REGIONAL DNIT GO/DF,2803.860107,5951.790039,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252
8,7801184,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,757/2019,8744139000151,G&E SERVICOS TERCEIRIZADOS LTDA,***.226.181-**,LUCIANO ALVES MOREIRA,...,SUPERINTENDENCIA REGIONAL DNIT GO/DF,1857.300049,4238.899902,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252
9,7801185,M.INFRAE,393011,SUPERINTENDENCIA REGIONAL DNIT-GO/DF,SUP.REG.GO/DF-DNIT,757/2019,8744139000151,G&E SERVICOS TERCEIRIZADOS LTDA,***.583.501-**,JANAINA LEMOS GAMA FREITAS,...,SUPERINTENDENCIA REGIONAL DNIT GO/DF,1962.699951,4343.790039,1,JANEIRO,2020,DNIT,DEPTO. NAC. DE INFRA-ESTRUTURA DE TRANSPORTES,39252,39252


In [10]:
df_2020.shape

(228724, 23)

In [13]:
df_2021 = pd.read_csv("../data/terceirizados/arquivos_tratados/contratos_consolidados_2021.csv", sep =",", low_memory=False)
df_2021.head()

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
0,7580056,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.118.217-**,EDILEUSA ALMEIDA DA SILVA,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2
1,7580057,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.568.397-**,JOSEFA DOS SANTOS,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2
2,7580058,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.386.217-**,LUZINETE DA SILVA PEREIRA,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2
3,7580059,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.169.494-**,JOSEILTON DA SILVA COSTA,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2
4,7580060,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.093.227-**,VERONICA PERRUCHO NUNES,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2
5,7580061,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.126.757-**,LUCIANE DA SILVA,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2
6,7580062,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.517.347-**,HILDA DOS SANTOS DA SILVA,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2
7,7580063,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.091.507-**,CARLOS MAURICIO VIANA DOS SANTOS,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2
8,7580064,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.684.627-**,RENATO ROCHA DO NASCIMENTO,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2
9,7580065,M.CIDADA,180080,MINISTERIO DA CIDADANIA - SECRETARIA ESPECIAL ...,AGLO,03/2017,5703030000188,CARDEAL GESTAO EMPRESARIAL E SERVICOS LTDA,***.265.667-**,KELIANE SANTOS DE SA,...,EGLO,1239.000000,4592.660156,1,JANEIRO,2021,MINIST. DO ESPORTE,MINISTERIO DO ESPORTE,51000,-2


In [12]:
df_2021.shape

(220032, 23)

In [14]:
df_2022 = pd.read_csv("../data/terceirizados/arquivos_tratados/contratos_consolidados_2022.csv", sep =",", low_memory=False)
df_2022.head()

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
0,8213079,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50226/2021,17865599000129,BRIGADA DE INCENDIO BH EIRELI,***.883.286-**,JEFFERSON HENRIQUE LOPES DE OLIVEIRA,...,DESEG,1756.709961,8125.000000,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
1,8213080,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50226/2021,17865599000129,BRIGADA DE INCENDIO BH EIRELI,***.464.036-**,EDMAURO FERREIRA CORREIA,...,DESEG,1756.709961,8125.000000,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
2,8213081,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50226/2021,17865599000129,BRIGADA DE INCENDIO BH EIRELI,***.806.696-**,VINICIUS AUGUSTO DOS REIS ARAUJO,...,DESEG,1756.709961,8125.000000,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
3,8213082,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50322/2021,31546484000798,CONFEDERAL VIGILANCIA E TRANSPORTE DE VALORES ...,***.047.856-**,JULIANO POSSIDONIO DA SILVA,...,DESEG,2260.820068,9584.759766,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
4,8213083,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50322/2021,31546484000798,CONFEDERAL VIGILANCIA E TRANSPORTE DE VALORES ...,***.152.976-**,ADAIR FRANCISCO DE FARIA,...,DESEG,2260.820068,8365.759766,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
5,8213084,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50322/2021,31546484000798,CONFEDERAL VIGILANCIA E TRANSPORTE DE VALORES ...,***.379.396-**,MARIO CESAR DA SILVA,...,DESEG,2260.820068,8365.759766,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
6,8213085,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50322/2021,31546484000798,CONFEDERAL VIGILANCIA E TRANSPORTE DE VALORES ...,***.451.889-**,PAULO ATAIDE DA SILVA,...,DESEG,2260.820068,9166.450195,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
7,8213086,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50322/2021,31546484000798,CONFEDERAL VIGILANCIA E TRANSPORTE DE VALORES ...,***.613.526-**,WELBERT TEIXEIRA LIMA DE ALMEIDA,...,DESEG,2260.820068,8365.759766,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
8,8213087,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50322/2021,31546484000798,CONFEDERAL VIGILANCIA E TRANSPORTE DE VALORES ...,***.897.196-**,DIEMERSON AGUIAR NEVES DE FARIA,...,DESEG,2260.820068,8365.759766,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
9,8213088,BACEN-OR,173044,BANCO CENTRAL DO BRASIL,BANCO CENTRAL-BH,50322/2021,31546484000798,CONFEDERAL VIGILANCIA E TRANSPORTE DE VALORES ...,***.233.926-**,CARLOS HENRIQUE DA SILVA,...,DESEG,2260.820068,9166.450195,1,JANEIRO,2022,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2


In [14]:
df_2022.shape

(229810, 23)

In [15]:
df_2023 = pd.read_csv("../data/terceirizados/arquivos_tratados/contratos_consolidados_2023.csv", sep =",", low_memory=False)
df_2023.head()

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
0,8443844,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.161.897-**,PAULO TARCISIO DA CONCEICAO BRAGA,...,DESEG,3193.250000,27304.939453,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
1,8443845,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.425.017-**,REGINALDO SANTOS MORAIS MONTEIRO,...,DESEG,3193.250000,23161.380859,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
2,8443846,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.681.217-**,CARLOS JOSE JACONIANO,...,DESEG,3193.250000,27304.939453,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
3,8443847,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.842.937-**,MARCIO BARBOZA GOMES,...,DESEG,3193.250000,27304.939453,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
4,8443848,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.370.397-**,VAGNER SANTOS DE SANT ANNA,...,DESEG,3193.250000,27304.939453,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
5,8443849,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.233.497-**,ALEX SANDRE CASTRO TEIXEIRA,...,DESEG,3193.250000,27304.939453,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
6,8443850,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.010.617-**,FERNANDO LUIS FREITAS MOREIRA,...,DESEG,3619.350098,22905.580078,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
7,8443851,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.531.567-**,ALEX SANDRO MARCIANO NUNES,...,DESEG,3193.250000,27304.939453,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
8,8443852,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.126.037-**,LEONARDO ROCHA DE ASSIS,...,DESEG,3193.250000,12088.070312,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2
9,8443853,BACEN-OR,173051,BANCO CENTRAL DO BRASIL,BACEN-R.JANEIRO,50400/2021,2060306000169,"FENIXX, VIGILANCIA, SEGURANCA E TRANSPORTE DE ...",***.113.777-**,RODRIGO SOARES VALENTIM,...,DESEG,3193.250000,27304.939453,1,JANEIRO,2023,BACEN-ORC.FIS/S.SOC,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201,-2


In [16]:
df_2023.shape

(233936, 23)

In [16]:
df_2024 = pd.read_csv("../data/terceirizados/arquivos_tratados/contratos_consolidados_2024.csv", sep =",", low_memory=False)
df_2024.head()

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
0,8679421,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,3325436000149,ADTEL FACILITIES LTDA,***.608.651-**,ALEXANDRO FABRICIO GOMES,...,CGLI,2238,5646,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
1,8679422,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,3325436000149,ADTEL FACILITIES LTDA,***.831.631-**,ADIELSON LOPES DE MELO,...,CGLI,1516,4536,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
2,8679423,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,6926324000131,ADTEL FACILITIES LTDA,***.316.751-**,THIAGO ROCHA OLIVEIRA,...,CGLI,1516,4536,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
3,8679424,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,6926324000131,ADTEL FACILITIES LTDA,***.957.551-**,MARIO CARDOSO DIAS,...,CGLI,1516,4536,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
4,8679425,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,6926324000131,ADTEL FACILITIES LTDA,***.458.031-**,CLEISON FARIAS VIEIRA,...,CGLI,7623,10695,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
5,8679426,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,6926324000131,ADTEL FACILITIES LTDA,***.657.301-**,WELLINGTON SOARES DA SILVA,...,CGLI,1516,4536,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
6,8679427,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,6926324000131,ADTEL FACILITIES LTDA,***.837.521-**,SAMUEL LUIZ DE OLIVEIRA GOMES,...,CGLI,2998,7314,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
7,8679428,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,6926324000131,ADTEL FACILITIES LTDA,***.081.121-**,MATHEUS CARNEIRO DO AMARAL SILVA,...,CGLI,1516,4536,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
8,8679429,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,6926324000131,ADTEL FACILITIES LTDA,***.487.266-**,MARCOS SAMUEL VAZ LEITE,...,CGLI,1516,4536,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
9,8679430,MAPA,130005,COORDENACAO GERAL DE LOGISTICA INSTITUCIONAL,CGOEF/DA/SE/MAPA,2023-10-01 00:00:00,6926324000131,ADTEL FACILITIES LTDA,***.540.181-**,JOSENCLEITON OLIVEIRA DAS VIRGENS,...,CGLI,2238,6013,1,JANEIRO,2024,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000


In [18]:
df_2024.shape

(156451, 23)

In [19]:
def converter_mes_para_numero(contrato):
    """
    Converte valores como 'jun/17' para '0000062017'.
    """
    
    # Dicionário para converter meses abreviados para números
    meses_dict = {
        "jan": "01", "fev": "02", "mar": "03", "abr": "04", "mai": "05", "jun": "06",
        "jul": "07", "ago": "08", "set": "09", "out": "10", "nov": "11", "dez": "12"
    }

    contrato = str(contrato).strip().lower()  # Converter para string e minúsculas

    match = re.match(r'([a-z]{3})/(\d{2,4})', contrato)  # Exemplo: jun/17 ou dez/2018
    if match:
        mes = match.group(1)  # Pega a parte do mês (ex: "jun")
        ano = match.group(2)  # Pega o ano (ex: "17" ou "2018")

        if mes in meses_dict:  # Converte mês para número
            mes_numerico = meses_dict[mes]

            # Ajusta o ano para formato completo (ex: "17" → "2017")
            if len(ano) == 2:
                ano = "20" + ano if int(ano) < 30 else "19" + ano  # Assume anos 2000+ até 2029, senão 1900+

            return f"000{mes_numerico}{ano}"  # Formato final

    return contrato  # Retorna original se não corresponder ao padrão

In [20]:
def limpar_nr_contrato(contrato):
    """
    Remove quaisquer caracteres especiais e mantém apenas números.
    Exemplo: '11.000/2016' → '110002016'
    """
    contrato = str(contrato).strip()  # Garantir que é string
    contrato = re.sub(r'\D', '', contrato)  # Remove tudo que não for número
    return contrato

In [21]:
# Aplicar a função de converter mês para número em todos os dataframes
df_2019['nr_contrato'] = df_2019['nr_contrato'].apply(converter_mes_para_numero)
df_2020['nr_contrato'] = df_2020['nr_contrato'].apply(converter_mes_para_numero)
df_2021['nr_contrato'] = df_2021['nr_contrato'].apply(converter_mes_para_numero)
df_2022['nr_contrato'] = df_2022['nr_contrato'].apply(converter_mes_para_numero)
df_2023['nr_contrato'] = df_2023['nr_contrato'].apply(converter_mes_para_numero)
df_2024['nr_contrato'] = df_2024['nr_contrato'].apply(converter_mes_para_numero)

In [22]:
# Aplicar a função de limpeza do número de contrato em todos os dataframes
df_2019['nr_contrato'] = df_2019['nr_contrato'].apply(limpar_nr_contrato)
df_2020['nr_contrato'] = df_2020['nr_contrato'].apply(limpar_nr_contrato)
df_2021['nr_contrato'] = df_2021['nr_contrato'].apply(limpar_nr_contrato)
df_2022['nr_contrato'] = df_2022['nr_contrato'].apply(limpar_nr_contrato)
df_2023['nr_contrato'] = df_2023['nr_contrato'].apply(limpar_nr_contrato)
df_2024['nr_contrato'] = df_2024['nr_contrato'].apply(limpar_nr_contrato)

In [23]:
df_2019['nr_contrato'].value_counts()

512017       6347
232014       5933
892017       5508
2232017      3762
182016       2767
             ... 
0232029         1
000112008       1
000022020       1
000022021       1
1682017         1
Name: nr_contrato, Length: 2276, dtype: int64

In [24]:
# Criar um dicionário com os DataFrames e seus respectivos anos
dfs_por_ano = {
    "2019": df_2019,
    "2020": df_2020,
    "2021": df_2021,
    "2022": df_2022,
    "2023": df_2023,
    "2024": df_2024
}

In [25]:
# Loop para verificar valores vazios em cada DataFrame
for ano, df in dfs_por_ano.items():
    print(f"\n🔍 Analisando df_{ano}:")

    # Verificar amostra de valores únicos
    print(f"Valores únicos antes do agrupamento (amostra): {df['nr_contrato'].unique()[:10]}")

    # Contar valores vazios ("") e NaN
    print(f"Total de valores vazios antes do agrupamento: {(df['nr_contrato'].str.strip() == '').sum()}")
    print(f"Total de valores NaN antes do agrupamento: {df['nr_contrato'].isna().sum()}")

    # Tipo de dado da coluna `nr_contrato`
    print(f"Tipo de dados de nr_contrato: {df['nr_contrato'].dtype}")

    # Exibir as primeiras linhas com valores vazios para análise
    df_vazios = df[df['nr_contrato'].str.strip() == ""]
    if not df_vazios.empty:
        print(f"\n📌 Linhas com `nr_contrato` vazio no ano {ano}:")
        display(df_vazios.head(10))  # Exibir apenas as primeiras 10 linhas para não poluir a saída


🔍 Analisando df_2019:
Valores únicos antes do agrupamento (amostra): ['012018' '022017' '022019' '032017' '292015' '392014' '942014'
 '030005002018' '030005002019' '030005002020']
Total de valores vazios antes do agrupamento: 9
Total de valores NaN antes do agrupamento: 0
Tipo de dados de nr_contrato: object

📌 Linhas com `nr_contrato` vazio no ano 2019:


,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
21455,7974600,M.INFRAE,393048,UNIDADE REGIONAL DE MINAS GERAIS - ANTT,URMG - ANTT,,00482840001029,LIDERANCA LIMPEZA E CONSERVACAO LTDA,***.577.336-**,EMANUELLE MARCIA DA LUZ CASTRO,...,COFIS/BELO HORIZONTE,522.3699951171875,1901.1800537109375,5,MAIO,2019,ANTT,AGENCIA NACIONAL DE TRANSPORTES TERRESTRES,39250,39250
57003,8043328,MIN.EDUC,158960,CAMPUS CAUCAIA,CAMPUS CAUCAIA,,08489384000160,ARTHOS SERVICOS E MANUTENCAO EIRELI (CNPJ 08.4...,***.360.733-**,MARCIO FERREIRA DA SILVA,...,CAMPUS CAUCAIA,1585.219970703125,3827.929931640625,5,MAIO,2019,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,26405
57407,8043390,MIN.EDUC,158966,CAMPUS PARACURU,IF DO CEARA,,21587512000176,REALIZA SERVICOS TERCEIRIZADOS EIRELI,***.642.393-**,FRANCISCA NADIA EVANGELISTA BALBINO,...,CAMPUS PARACURU,1133.1700439453125,3055.840087890625,5,MAIO,2019,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,-2
83267,8057786,MJSP,200356,SUPERINTENDENCIA REGIONAL DE POLICIA FEDERAL N...,SR/DPF/RJ,,78533312000158,PLANSUL PLANEJAMENTO E CONSULTORIA EIRELI,***.934.897-**,ESTEVAO NASCIMENTO DA SILVA,...,DPF/NIG/RJ,1194.0,3338.919921875,5,MAIO,2019,DPF,DEPARTAMENTO DE POLICIA FEDERAL,30108,20115
185401,7165670,MTPAC,393048,UNIDADE REGIONAL DE MINAS GERAIS - ANTT,URMG - ANTT,,00482840001029,LIDERANCA LIMPEZA E CONSERVACAO LTDA,***.577.336-**,EMANUELLE MARCIA DA LUZ CASTRO,...,COFIS/BELO HORIZONTE,"522,37","1901,18",1,JANEIRO,2019,ANTT,AGENCIA NACIONAL DE TRANSPORTES TERRESTRES,39250,39250
215074,7916992,MIN.AGRI,130021,"SUPERINTENDENCIA FEDERAL DE AGGRICULTURA, PECU...",SFA/PI/MAPA,,09540692000135,BETA BRASIL SERVICOS DE CONSERVACAO E LIMPEZA ...,***.030.303-**,WASHINGTON LIMA SILVA,...,SEAD,1959.0400390625,3785.909912109375,9,SETEMBRO,2019,MIN.AGRICULTURA,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",22000,13000
245753,7951789,MIN.EDUC,158516,"INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNO...",IF SC,,13237931000150,SOLUCOES LOOPING LTDA ME,***.394.329-**,SERGIO DORNELES DE MOURA,...,CHAPECO,1321.280029296875,4225.0,9,SETEMBRO,2019,IF SC,"INST.FED.DE EDUC.,CIENC.E TEC.DE STA.CATARINA",26438,26438
262936,7953760,MIN.EDUC,158966,CAMPUS PARACURU,IF DO CEARA,,21587512000176,REALIZA SERVICOS TERCEIRIZADOS EIRELI,***.642.393-**,FRANCISCA NADIA EVANGELISTA BALBINO,...,CAMPUS PARACURU,1133.1700439453125,3055.840087890625,9,SETEMBRO,2019,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,-2
267648,7939607,MIN.EDUC,154069,FUNDACAO UNIVERSIDADE FEDERAL DE SAO JOAO DEL-REI,UFSJ,,08764312000183,MAXIMA SERVICOS E OBRAS LTDA,***.169.976-**,LUCAS HENRIQUE DOS SANTOS,...,SEDE/CSA,1440.050048828125,3229.75,9,SETEMBRO,2019,UFSJ,FUNDACAO UNIVERSIDADE FEDERAL DE S.J.DEL-REI,26285,26285



🔍 Analisando df_2020:
Valores únicos antes do agrupamento (amostra): ['6232017' '7572019' '012018' '2522017' '2562015' '022018' '0332018'
 '2572015' '2582014' '2582015']
Total de valores vazios antes do agrupamento: 73
Total de valores NaN antes do agrupamento: 0
Tipo de dados de nr_contrato: object

📌 Linhas com `nr_contrato` vazio no ano 2020:


,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
8715,7802322,M.INFRAE,393048,UNIDADE REGIONAL DE MINAS GERAIS - ANTT,URMG - ANTT,,482840001029,LIDERANCA LIMPEZA E CONSERVACAO LTDA,***.577.336-**,EMANUELLE MARCIA DA LUZ CASTRO,...,COFIS/BELO HORIZONTE,522.369995,1901.180054,1,JANEIRO,2020,ANTT,AGENCIA NACIONAL DE TRANSPORTES TERRESTRES,39250,39250
61800,7856951,MIN.EDUC,158504,INSTITUTO FEDERAL FARROUPILHA CAMPUS SANTA ROSA,IFFAR/CAMPUS S.ROSA,,17453147000130,IGUACU DESENVOLVIMENTO LTDA,***.247.840-**,LUIZ ARIEL DE OLIVEIRA TIBOLA,...,CAMPUS SANTA ROSA,1287.849976,2981.120117,1,JANEIRO,2020,IF FARROUPILHA,"INST.FED.DE EDUC.,CIENC.E TEC.FARROUPILHA",26420,26420
76346,7853987,MIN.EDUC,158358,IFPI - ANGICAL,IFPI/CAMPUS ANGICAL,,12628143000121,RH SERVICOS,***.249.323-**,GONCALO PEREIRA DA SILVA,...,CAANG,1369.489990,2984.979980,1,JANEIRO,2020,IF DO PIAUI,"INST.FED.DE EDUC.,CIENC.E TEC.DO PIAUI",26431,26431
76831,7859008,MIN.EDUC,158966,CAMPUS PARACURU,IF DO CEARA,,21587512000176,REALIZA SERVICOS TERCEIRIZADOS EIRELI,***.642.393-**,FRANCISCA NADIA EVANGELISTA BALBINO,...,CAMPUS PARACURU,1133.170044,3055.840088,1,JANEIRO,2020,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,-2
78296,7854300,MIN.EDUC,158368,IFRN-NATAL ZONA NORTE,IFRN/CAMPUS NATAL Z,,3116706000101,TALIMPO LIMPEZA URBANA LTDA.,***.955.944-**,ABI PAULA RODRIGUES DA SILVA,...,NATAL - ZONA NORTE,1040.000000,3010.409912,1,JANEIRO,2020,IF DO RN,"INST.FED.DE EDUC.,CIENC.E TEC.DO RN",26435,26435
93926,7722250,M.INFRAE,393048,UNIDADE REGIONAL DE MINAS GERAIS - ANTT,URMG - ANTT,,482840001029,LIDERANCA LIMPEZA E CONSERVACAO LTDA,***.577.336-**,EMANUELLE MARCIA DA LUZ CASTRO,...,COFIS/BELO HORIZONTE,522.369995,1901.180054,5,MAIO,2020,ANTT,AGENCIA NACIONAL DE TRANSPORTES TERRESTRES,39250,39250
129300,7780152,MIN.EDUC,158966,CAMPUS PARACURU,IF DO CEARA,,21587512000176,REALIZA SERVICOS TERCEIRIZADOS EIRELI,***.642.393-**,FRANCISCA NADIA EVANGELISTA BALBINO,...,CAMPUS PARACURU,1224.550049,3140.510010,5,MAIO,2020,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,-2
135664,7771215,MIN.EDUC,155624,PEDRO II - HUMAITA I - PORTEIRO,CAMPUS HUMAITA I,,12978986000158,TS LOCACAO DE MAO DE OBRA EM GERAL EIRELI,***.917.957-**,LUIZ FERNANDO ALMEIDA COSTA,...,COLEGIO PEDRO II,1166.660034,3074.919922,5,MAIO,2020,COLEGIO PEDRO II,COLEGIO PEDRO II,26201,-2
135665,7771216,MIN.EDUC,155624,PEDRO II - HUMAITA I - PORTEIRO,CAMPUS HUMAITA I,,12978986000158,TS LOCACAO DE MAO DE OBRA EM GERAL EIRELI,***.820.727-**,DAVI ANTONIO DA SILVA JUNIOR,...,COLEGIO PEDRO II,1166.660034,3074.919922,5,MAIO,2020,COLEGIO PEDRO II,COLEGIO PEDRO II,26201,-2
135908,7771251,MIN.EDUC,155626,PEDRO II - REALENGO - PORTEIROS,CAMPUS REALENGO I,,12978986000158,TS LOCACAO DE MAO DE OBRA EM GERAL EIRELI,***.718.567-**,VINICIUS BADU DA SILVA,...,COLEGIO PEDRO II,1166.660034,3074.919922,5,MAIO,2020,COLEGIO PEDRO II,COLEGIO PEDRO II,26201,-2



🔍 Analisando df_2021:
Valores únicos antes do agrupamento (amostra): ['032017' '052017' '072017' '332020' '322018' '112018' '352020' '022019'
 '0052016' '0132020']
Total de valores vazios antes do agrupamento: 11
Total de valores NaN antes do agrupamento: 0
Tipo de dados de nr_contrato: object

📌 Linhas com `nr_contrato` vazio no ano 2021:


...

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
4572,8245201,MIN.EDUC,158133,REITORIA,IF DO CEARA,,7917508000106,PERES SERVICOS DE SEGURANCA LTDA,***.674.173-**,CAIO ATHISSON SOUZA DOS SANTOS,...,REITORIA,2250.570068,5253.200195,1,JANEIRO,2022,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,26405
12831,8251153,MIN.EDUC,158966,CAMPUS PARACURU,IF DO CEARA,,21587512000176,REALIZA SERVICOS TERCEIRIZADOS EIRELI,***.642.393-**,FRANCISCA NADIA EVANGELISTA BALBINO,...,CAMPUS PARACURU,1224.550049,3140.510010,1,JANEIRO,2022,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,-2
13123,8251191,MIN.EDUC,158969,CAMPUS AVANCADO DE ACOPIARA,NaN,,14292203000103,ACESSO SEGURANCA PRIVADA LTDA - ME,***.801.233-**,RENATO ARAUJO DE MELO,...,CAMPUS AVANCADO DE ACOPIARA,1840.930054,4067.760010,1,JANEIRO,2022,<N/I>,<N/I>,-2,-2
16788,8246677,MIN.EDUC,158314,CAMPUS LIMOEIRO DO NORTE,IFCE/CAMPUS L. NORT,,3340389000102,JPA SERVICOS COMBINADOS LTDA ME,***.584.633-**,RAIMUNDO ERINEUDO DA SILVA FREITAS,...,CAMPUS LIMOEIRO DO NORTE,1100.000000,2743.280029,1,JANEIRO,2022,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,26405
16789,8246678,MIN.EDUC,158314,CAMPUS LIMOEIRO DO NORTE,IFCE/CAMPUS L. NORT,,3340389000102,JPA SERVICOS COMBINADOS LTDA ME,***.891.583-**,LAIZ KELLY OLIVEIRA DA SILVA,...,CAMPUS LIMOEIRO DO NORTE,1100.000000,2743.280029,1,JANEIRO,2022,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,26405
19992,8247082,MIN.EDUC,158322,CAMPUS ACARAU,IFCE/CAMPUS ACARAU,,5559732000130,CRESCER SERVICOS LTDA,***.661.303-**,MARLENE DOS SANTOS OLIVEIRA,...,CAMPUS ACARAU,1146.780029,3069.699951,1,JANEIRO,2022,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,26405
20153,8247112,MIN.EDUC,158323,CAMPUS CANINDE,IFCE/CAMPUS CANINDE,,7990965000118,AGAPE CONSTRUCOES E SERVICOS LTDA,***.108.683-**,FRANCISCO JALES DO NASCIMENTO SILVA,...,CAMPUS CANINDE,1280.869995,3902.040039,1,JANEIRO,2022,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,26405
20154,8247113,MIN.EDUC,158323,CAMPUS CANINDE,IFCE/CAMPUS CANINDE,,7990965000118,AGAPE CONSTRUCOES E SERVICOS LTDA,***.235.453-**,MARIA TAMYRIS ALVES BARROS,...,CAMPUS CANINDE,1280.869995,3902.040039,1,JANEIRO,2022,IF DO CEARA,"INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA",26405,26405
77662,8315100,MIN.EDUC,154046,UNIVERSIDADE FEDERAL DE OURO PRETO,UFOP,,1696924000137,REDENTOR SEGURANCA E VIGILANCIA EIRELI,***.159.746-**,ALEXANDRE AUGUSTO GAMA,...,JOAO MONLEVARDE,2062.389893,7076.470215,5,MAIO,2022,UFOP,UNIVERSIDADE FEDERAL DE OURO PRETO,26277,26277
77663,8315101,MIN.EDUC,154046,UNIVERSIDADE FEDERAL DE OURO PRETO,UFOP,,1696924000137,REDENTOR SEGURANCA E VIGILANCIA EIRELI,***.581.166-**,RICARDO MALTILDES COSTA,...,MARIANA,2062.389893,6123.029785,5,MAIO,2022,UFOP,UNIVERSIDADE FEDERAL DE OURO PRETO,26277,26277



🔍 Analisando df_2023:
Valores únicos antes do agrupamento (amostra): ['504002021' '504692021' '505352018' '112018' '112020' '506862020'
 '122020' '52412020' '53282018' '53342020']
Total de valores vazios antes do agrupamento: 24
Total de valores NaN antes do agrupamento: 0
Tipo de dados de nr_contrato: object

📌 Linhas com `nr_contrato` vazio no ano 2023:


,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
27356,8484262,MIN.EDUC,158363,INSTITUTO FEDERAL DE EDUCACAOCIENCIA E TECNOLO...,IFPI/CAMPUS URUCUI,,18089589000101,G KELLY DA SILVA ARAUJO & CIA LTDA,***.046.913-**,FRANCISCO SOBRINHO DE JESUS,...,IFPI CAURU,1386.550049,3217.899902,1,JANEIRO,2023,IF DO PIAUI,"INST.FED.DE EDUC.,CIENC.E TEC.DO PIAUI",26431,26431
51683,8499724,MJSP,200119,SUPERINTENDENCIA DA PRF NO RIO GRANDE DO SUL,SRPRF-RS,,3315120000176,BENETTON SERVICOS DE TERCEIRIZACAO LTDA,***.464.480-**,MAGDA SUZANA ROSADO DE SOUZA,...,GOC/RS,1352.979980,3414.429932,1,JANEIRO,2023,DPRF/MJ,DEPARTAMENTO DE POLICIA RODOVIARIA FEDERAL/MJ,30802,30802
62064,8501103,MMA,440001,"SUBSECRET.DE PLANEJ.,ORC.E ADMINIST.­SPOA/MMA",CGFC/SPOA/SECEX/MMA,,27704075000100,PROFORCE TERCEIRIZACOES E SERVICOS EIRELI,***.258.061-**,JOAO PAULO ALVES MADEIRA DA SILVA,...,MMA,3973.840088,8402.009766,1,JANEIRO,2023,MIN. MEIO AMBIENTE,MINISTERIO DO MEIO AMBIENTE,44000,40111
62065,8501104,MMA,440001,"SUBSECRET.DE PLANEJ.,ORC.E ADMINIST.­SPOA/MMA",CGFC/SPOA/SECEX/MMA,,27704075000100,PROFORCE TERCEIRIZACOES E SERVICOS EIRELI,***.603.201-**,ELTON EDILON VIEIRA DA SILVA,...,MMA,3973.840088,8402.009766,1,JANEIRO,2023,MIN. MEIO AMBIENTE,MINISTERIO DO MEIO AMBIENTE,44000,40111
62066,8501105,MMA,440001,"SUBSECRET.DE PLANEJ.,ORC.E ADMINIST.­SPOA/MMA",CGFC/SPOA/SECEX/MMA,,27704075000100,PROFORCE TERCEIRIZACOES E SERVICOS EIRELI,***.870.123-**,EVANEY SOARES ARAUJO,...,MMA,1416.750000,4960.419922,1,JANEIRO,2023,MIN. MEIO AMBIENTE,MINISTERIO DO MEIO AMBIENTE,44000,40111
62067,8501106,MMA,440001,"SUBSECRET.DE PLANEJ.,ORC.E ADMINIST.­SPOA/MMA",CGFC/SPOA/SECEX/MMA,,27704075000100,PROFORCE TERCEIRIZACOES E SERVICOS EIRELI,***.620.481-**,MARIANA RODRIGUES BRAGA DE SOUSA,...,MMA,3936.419922,8311.950195,1,JANEIRO,2023,MIN. MEIO AMBIENTE,MINISTERIO DO MEIO AMBIENTE,44000,40111
62068,8501107,MMA,440001,"SUBSECRET.DE PLANEJ.,ORC.E ADMINIST.­SPOA/MMA",CGFC/SPOA/SECEX/MMA,,27704075000100,PROFORCE TERCEIRIZACOES E SERVICOS EIRELI,***.613.141-**,MARIA DO CARMO DE QUEIROZ,...,MMA,2832.969971,4960.419922,1,JANEIRO,2023,MIN. MEIO AMBIENTE,MINISTERIO DO MEIO AMBIENTE,44000,40111
62069,8501108,MMA,440001,"SUBSECRET.DE PLANEJ.,ORC.E ADMINIST.­SPOA/MMA",CGFC/SPOA/SECEX/MMA,,27704075000100,PROFORCE TERCEIRIZACOES E SERVICOS EIRELI,***.342.831-**,JULIO SALES GOMES DA SILVA,...,MMA,3936.419922,8311.950195,1,JANEIRO,2023,MIN. MEIO AMBIENTE,MINISTERIO DO MEIO AMBIENTE,44000,40111
62070,8501109,MMA,440001,"SUBSECRET.DE PLANEJ.,ORC.E ADMINIST.­SPOA/MMA",CGFC/SPOA/SECEX/MMA,,27704075000100,PROFORCE TERCEIRIZACOES E SERVICOS EIRELI,***.753.721-**,HEGLISSON COUTO PINTO,...,MMA,2832.969971,4960.419922,1,JANEIRO,2023,MIN. MEIO AMBIENTE,MINISTERIO DO MEIO AMBIENTE,44000,40111
62071,8501110,MMA,440001,"SUBSECRET.DE PLANEJ.,ORC.E ADMINIST.­SPOA/MMA",CGFC/SPOA/SECEX/MMA,,27704075000100,PROFORCE TERCEIRIZACOES E SERVICOS EIRELI,***.006.971-**,TIAGO ZICA DE CARVALHO,...,MMA,2832.969971,4960.419922,1,JANEIRO,2023,MIN. MEIO AMBIENTE,MINISTERIO DO MEIO AMBIENTE,44000,40111



🔍 Analisando df_2024:
Valores únicos antes do agrupamento (amostra): ['20231001000000' '542020' '0832020' '2012020' '692023' '0306000102023'
 '030600202023' '030600202024' '030600302023' '20201201000000']
Total de valores vazios antes do agrupamento: 184
Total de valores NaN antes do agrupamento: 0
Tipo de dados de nr_contrato: object

📌 Linhas com `nr_contrato` vazio no ano 2024:


,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
9852,8707500,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.348.077-**,SOLANGE MARIA VICENTE,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245
9853,8707501,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.463.507-**,VALERIA AZEVEDO DE LIMA,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245
9854,8707502,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.481.757-**,ADRIANA DA CONCEICAO,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245
9855,8707503,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.494.557-**,FABIO DA CONCEICAO MARTINS,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245
9856,8707504,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.203.977-**,CRISTIANE DOS SANTOS PEREIRA,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245
9857,8707505,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.104.728-**,GILVANETE ASSIS DE OLIVEIRA,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245
9858,8707506,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.194.914-**,AUZENI AZEVEDO DOS SANTOS,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245
9859,8707507,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.071.924-**,MARIA DE FATIMA PEDRO DUARTE,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245
9860,8707508,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.584.307-**,ERONICE ROCHA,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245
9861,8707509,MIN.EDUC,153115,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ,UFRJ,,36455657000190,LIPA SERVICOS GERAIS LTDA,***.791.907-**,ALBERTO DA SILVA,...,HESFA,1516,4708,1,JANEIRO,2024,UFRJ,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,26245,26245


In [26]:
# Veja que as informações que têm aqui não são suficientes para identificar qual é o contrato, uma vez que só com essa empresa há 8 contratos diferentes, 
# não há como inferir 100% com certeza
df_2024.iloc[9852]

id_terc                                                                8707500
sg_orgao_sup_tabela_ug                                                MIN.EDUC
cd_ug_gestora                                                           153115
nm_ug_tabela_ug                  UNIVERSIDADE FEDERAL DO RIO DE JANEIRO – UFRJ
sg_ug_gestora                                                             UFRJ
nr_contrato                                                                   
nr_cnpj                                                         36455657000190
nm_razao_social                                      LIPA SERVICOS GERAIS LTDA
nr_cpf                                                          ***.348.077-**
nm_terceirizado                                          SOLANGE MARIA VICENTE
nm_categoria_profissional    514230 - COLETOR DE RESIDUOS SOLIDOS DE SERVIC...
nm_escolaridade                                    ENSINO FUNDAMENTAL COMPLETO
nr_jornada                                          

In [27]:
# A escolha metodológica é retirar as linhas com nr_contrato vazio para não impactar na validação
# Remover as linhas onde `nr_contrato` está vazio
for ano, df in dfs_por_ano.items():
    print(f"\n📌 Antes da limpeza - df_{ano}: {len(df)} linhas")
    
    # Filtrar as linhas onde `nr_contrato` NÃO está vazio
    dfs_por_ano[ano] = df[df['nr_contrato'].str.strip() != ""]
    
    print(f"✅ Após a limpeza - df_{ano}: {len(dfs_por_ano[ano])} linhas restantes")

df_2019, df_2020, df_2021, df_2022, df_2023, df_2024 = dfs_por_ano.values()


📌 Antes da limpeza - df_2019: 276554 linhas
✅ Após a limpeza - df_2019: 276545 linhas restantes

📌 Antes da limpeza - df_2020: 228724 linhas
✅ Após a limpeza - df_2020: 228651 linhas restantes

📌 Antes da limpeza - df_2021: 220032 linhas
✅ Após a limpeza - df_2021: 220021 linhas restantes

📌 Antes da limpeza - df_2022: 229810 linhas
✅ Após a limpeza - df_2022: 229771 linhas restantes

📌 Antes da limpeza - df_2023: 233936 linhas
✅ Após a limpeza - df_2023: 233912 linhas restantes

📌 Antes da limpeza - df_2024: 156451 linhas
✅ Após a limpeza - df_2024: 156267 linhas restantes


In [28]:
# Criar um dataframe único para validação dos dados
dfs_terceirizados = [df_2019, df_2020, df_2021, df_2022, df_2023, df_2024]

# Concatenar todos os anos em um único DataFrame
df_total = pd.concat(dfs_terceirizados, ignore_index=True)

# Remover linhas 100% idênticas
df_total = df_total.drop_duplicates()

# Exibir o número total de registros após a limpeza
print(f"✅ DataFrame único consolidado, com {len(df_total)} registros após remoção de duplicatas.")

✅ DataFrame único consolidado, com 1345167 registros após remoção de duplicatas.


In [30]:
#df_total.to_csv("../data/terceirizados/arquivos_tratados/df_total.csv", index=False, encoding="utf-8")

## Padronizar o número do contrato para ter 9 dígitos, conforme API

In [2]:
df_total = pd.read_csv("../data/terceirizados/arquivos_tratados/df_total.csv", low_memory=False)

In [3]:
df_total.columns

Index(['id_terc', 'sg_orgao_sup_tabela_ug', 'cd_ug_gestora', 'nm_ug_tabela_ug',
       'sg_ug_gestora', 'nr_contrato', 'nr_cnpj', 'nm_razao_social', 'nr_cpf',
       'nm_terceirizado', 'nm_categoria_profissional', 'nm_escolaridade',
       'nr_jornada', 'nm_unidade_prestacao', 'vl_mensal_salario',
       'vl_mensal_custo', 'Num_Mes_Carga', 'Mes_Carga', 'Ano_Carga',
       'sg_orgao', 'nm_orgao', 'cd_orgao_siafi', 'cd_orgao_siape'],
      dtype='object')

In [4]:
df_total['nr_contrato'].isna().value_counts()

False    1345167
Name: nr_contrato, dtype: int64

In [5]:
# Função para garantir que nr_contrato tenha exatamente 9 dígitos
def formatar_nr_contrato(nr_contrato):
    nr_contrato = str(nr_contrato).strip()  # Remover espaços extras

    # Se já tiver 9 dígitos, remover zeros à esquerda e preencher novamente para garantir 9 dígitos
    if len(nr_contrato) == 9:
        return nr_contrato.lstrip('0').zfill(9)
    
    # Se tiver menos de 9 dígitos, adicionar zeros à esquerda
    return nr_contrato.zfill(9)

# Aplicando a função à coluna nr_contrato
df_total['nr_contrato'] = df_total['nr_contrato'].apply(formatar_nr_contrato)

In [6]:
df_total['nr_contrato'].isna().value_counts()

False    1345167
Name: nr_contrato, dtype: int64

## Corrigir e validar código de SIAFI

In [7]:
df_total[df_total['cd_orgao_siafi'] == -2]

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
57985,8043464,MIN.EDUC,158973,CAMPUS MARANGUAPE,NaN,000192018,10732146000185,SR TERCEIRIZACOES EIRELI - ME,***.037.033-**,ANTONIO RIVELINO FERREIRA,...,CAMPUS MARANGUAPE,1014.5,3089.47998046875,5,MAIO,2019,<N/I>,<N/I>,-2,-2
57986,8043465,MIN.EDUC,158973,CAMPUS MARANGUAPE,NaN,000192018,10732146000185,SR TERCEIRIZACOES EIRELI - ME,***.383.673-**,IRAILSON SILVA COELHO,...,CAMPUS MARANGUAPE,1014.5,3089.47998046875,5,MAIO,2019,<N/I>,<N/I>,-2,-2
57987,8043466,MIN.EDUC,158973,CAMPUS MARANGUAPE,NaN,000192018,10732146000185,SR TERCEIRIZACOES EIRELI - ME,***.443.153-**,ANA FABIA DA COSTA FERREIRA,...,CAMPUS MARANGUAPE,1014.5,3089.47998046875,5,MAIO,2019,<N/I>,<N/I>,-2,-2
57988,8043467,MIN.EDUC,158973,CAMPUS MARANGUAPE,NaN,000192018,10732146000185,SR TERCEIRIZACOES EIRELI - ME,***.014.673-**,ROSANA AQUINO DO NASCIMENTO,...,CAMPUS MARANGUAPE,1014.5,3089.47998046875,5,MAIO,2019,<N/I>,<N/I>,-2,-2
57989,8043468,MIN.EDUC,158973,CAMPUS MARANGUAPE,NaN,000362018,14292203000103,ACESSO SEGURANCA PRIVADA LTDA - ME,***.722.443-**,ALEXSANDRO DE OLIVEIRA LIMA,...,CAMPUS MARANGUAPE,2158.5400390625,4405.56982421875,5,MAIO,2019,<N/I>,<N/I>,-2,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319147,8792281,MIN.EDUC,156690,INST. FED. DE EDUC. CIENC. E TEC. DA PARAIBA -...,NaN,000312022,10926785000181,PRONTSERV PRESTACAO DE SERVICOS DE LIMPEZA EIRELI,***.445.164-**,ALDENIRA DOS SANTOS VILAR,...,IFPB CAMPUS SANTA LUZIA,1805,4089,5,MAIO,2024,<N/I>,<N/I>,-2,-2
1319148,8792282,MIN.EDUC,156690,INST. FED. DE EDUC. CIENC. E TEC. DA PARAIBA -...,NaN,000312022,10926785000181,PRONTSERV PRESTACAO DE SERVICOS DE LIMPEZA EIRELI,***.930.204-**,RAYANE SANTOS LIMA,...,IFPB CAMPUS SANTA LUZIA,1322,3913,5,MAIO,2024,<N/I>,<N/I>,-2,-2
1319149,8792283,MIN.EDUC,156690,INST. FED. DE EDUC. CIENC. E TEC. DA PARAIBA -...,NaN,000312022,10926785000181,PRONTSERV PRESTACAO DE SERVICOS DE LIMPEZA EIRELI,***.931.794-**,ELIENE CARNEIRO SATURNINO,...,IFPB CAMPUS SANTA LUZIA,1322,3913,5,MAIO,2024,<N/I>,<N/I>,-2,-2
1319150,8792284,MIN.EDUC,156690,INST. FED. DE EDUC. CIENC. E TEC. DA PARAIBA -...,NaN,000312022,10926785000181,PRONTSERV PRESTACAO DE SERVICOS DE LIMPEZA EIRELI,***.295.194-**,CAMILA CAIANE DA SILVA,...,IFPB CAMPUS SANTA LUZIA,1322,3913,5,MAIO,2024,<N/I>,<N/I>,-2,-2


In [8]:
df_total[(df_total['nm_ug_tabela_ug'] == 'SECRETARIA GERAL DE INFORMATICA - SIN') & (df_total['cd_orgao_siafi'] == -2)]['sg_orgao_sup_tabela_ug'].unique()

array(['MIN.EDUC'], dtype=object)

In [9]:
# Perceba que o nm_unidade_prestacao tem uma variedade de informação
df_total[(df_total['nm_ug_tabela_ug'] == 'MINISTERIO DA ECONOMIA') & (df_total['cd_orgao_siafi'] == -2)]['nm_unidade_prestacao'].unique()

array(['CGEAD-COASP', 'CGEAD', 'CGTIP', 'OUVIDORIA DA ECONOMIA', 'SIC',
       'CGEAD-SETEL', 'MINISTERIO DA ECONOMIA', 'BLOCO K', 'CARF-COGEC',
       'COANA/SRF', 'COPOL/SRF', 'CAC/SRF', 'SEFAZ/SECAP', 'SEPRT',
       'CORREGEDORIA/SE', 'SEPRT/SPREV/SRGPS', 'SEFAZ/CONFAZ',
       'SEPEC/SPPE', 'SRF', 'DGP-SATEC/SE', 'PGFN', 'DRJ-SRF',
       'SEGES/SEDGG', 'COPES-SRF', 'SEDDM',
       'SEFAZ/STN/NUINF/CODIN-ARQUIVO', 'COGEP-SRF', 'DEPRO/SGP/SEDGG',
       'PRFN 1/DIDE1', 'SRF/ALFANGEGA', 'SEDGG', 'PRFN 1/DIDAU',
       'CORREGEDORIA', 'SEFAZ/SOF', 'CARF-CEGAP', 'SEFAZ/DEF',
       'COAD/SEGES/SEDGG', 'SEFAZ/STN', 'SEFAZ/GAB', 'SEFAZ/CRSFN',
       'SECINT/CAMEX', 'DIAM-SRF', 'COTEC/SRF',
       'SGC/DGP/CGCOM/DCAPE/CAPE/SE', 'SEAE/SEPEC', 'PRFN 1/DIGAB',
       'DECIPEX/SGP/SEDGG', 'SEFAZ', 'SEPEC', 'SEASS/CGPAQ/DGP/SE',
       'DTI/SE', 'SEPRT/SPREV/SPMF', 'SEDGG/GAB', 'CARF/COSUP/SEPAJ',
       'CARF', 'SEPRT/SPREV/SRPPS', 'PRFN 1/DIDE2', 'SEEFAZ', 'CGLIC',
       'DGP/SE', 'SGC/SE

In [10]:
# Corrigir cd_orgao_siafi para MGI e ME dentro do filtro específico
df_total.loc[(df_total['nm_ug_tabela_ug'] == 'MINISTERIO DA ECONOMIA') & 
              (df_total['cd_orgao_siafi'] == -2) & 
              (df_total['sg_orgao_sup_tabela_ug'] == 'MGI'), 'cd_orgao_siafi'] = 46000

# Por uma escolha metodológica, também vamos atribuir o código SIAFI do MGI, uma vez que o COLABORAGOV é um guarda-chuva para mais de 12 ministérios.
# A chance de acerto aumenta, uma vez que é difícil identificar por nm_unidade_prestacao
df_total.loc[(df_total['nm_ug_tabela_ug'] == 'MINISTERIO DA ECONOMIA') & 
              (df_total['cd_orgao_siafi'] == -2) & 
              (df_total['sg_orgao_sup_tabela_ug'] == 'ME.'), 'cd_orgao_siafi'] = 46000

In [11]:
df_total[(df_total['nm_ug_tabela_ug'] == 'SECRETARIA GERAL DE INFORMATICA') & (df_total['cd_orgao_siafi'] == -2)]

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
481172,7695362,MIN.EDUC,156403,SECRETARIA GERAL DE INFORMATICA,NaN,000032019,5637990000197,BELMAX SERVICOS DE LIMPEZA LTDA,***.137.568-**,IVANE DAS MERCES OLIVEIRA CABRERA,...,UFSCAR - SAO CARLOS,1298.9200439453125,3383.0,9,SETEMBRO,2020,<N/I>,<N/I>,-2,-2
481173,7695363,MIN.EDUC,156403,SECRETARIA GERAL DE INFORMATICA,NaN,000032019,5637990000197,BELMAX SERVICOS DE LIMPEZA LTDA,***.287.068-**,ISABELA CAROLINE HILARIO,...,UFSCAR - SAO CARLOS,1298.9200439453125,3383.0,9,SETEMBRO,2020,<N/I>,<N/I>,-2,-2
481174,7695364,MIN.EDUC,156403,SECRETARIA GERAL DE INFORMATICA,NaN,000052019,3399966000131,TECHCOM TECNOLOGIA E INFORMATICA EIRELI,***.990.081-**,WILLIAN DE PAULO RODRIGUES FRIAS,...,UFSCAR - SAO CARLOS,1653.530029296875,3759.93994140625,9,SETEMBRO,2020,<N/I>,<N/I>,-2,-2
481175,7695365,MIN.EDUC,156403,SECRETARIA GERAL DE INFORMATICA,NaN,000052019,3399966000131,TECHCOM TECNOLOGIA E INFORMATICA EIRELI,***.711.025-**,WELITON SANTOS DE BRITO,...,UFSCAR - SAO CARLOS,1653.530029296875,3735.85009765625,9,SETEMBRO,2020,<N/I>,<N/I>,-2,-2
481176,7695366,MIN.EDUC,156403,SECRETARIA GERAL DE INFORMATICA,NaN,000052019,3399966000131,TECHCOM TECNOLOGIA E INFORMATICA EIRELI,***.580.088-**,RAFAEL BATISTA MURASAKI,...,UFSCAR - SAO CARLOS,661.4099731445312,1503.97998046875,9,SETEMBRO,2020,<N/I>,<N/I>,-2,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317609,8792124,MIN.EDUC,156403,SECRETARIA GERAL DE INFORMATICA,NaN,000192022,8312139000182,ELO ADMINISTRACAO & TERCEIRIZACAO EIRELI,***.137.568-**,HELOISA DA SILVA LIBERATO DOMINGOS,...,SECRETARIA GERAL DE INFORMATICA,1590,4161,5,MAIO,2024,<N/I>,<N/I>,-2,-2
1317610,8792125,MIN.EDUC,156403,SECRETARIA GERAL DE INFORMATICA,NaN,000192022,8312139000182,ELO ADMINISTRACAO & TERCEIRIZACAO EIRELI,***.213.678-**,VICTOR HUGO PASTEGA DE TOLEDO,...,COORDENADORIA DOS PROGRAMAS DE IC E TECNOLOGICA,1590,4161,5,MAIO,2024,<N/I>,<N/I>,-2,-2
1317611,8792126,MIN.EDUC,156403,SECRETARIA GERAL DE INFORMATICA,NaN,000192022,8312139000182,ELO ADMINISTRACAO & TERCEIRIZACAO EIRELI,***.287.068-**,ISABELA CAROLINE HILARIO,...,SECRETARIA GERAL DE INFORMATICA,1590,4161,5,MAIO,2024,<N/I>,<N/I>,-2,-2
1317612,8792127,MIN.EDUC,156403,SECRETARIA GERAL DE INFORMATICA,NaN,000192022,8312139000182,ELO ADMINISTRACAO & TERCEIRIZACAO EIRELI,***.306.238-**,JULIE GABRIELI HILARIO,...,PRO-REITORIA DE GESTAO DE PESSOAS,1590,4161,5,MAIO,2024,<N/I>,<N/I>,-2,-2


In [12]:
df_siafi_negativo = df_total[df_total['cd_orgao_siafi'] == -2].groupby('nm_ug_tabela_ug').size().reset_index(name='quantidade')

In [13]:
df_siafi_negativo

,nm_ug_tabela_ug,quantidade
0,CAMPUS ACOPIARA,73
1,CAMPUS AVANCADO DE ACOPIARA,118
2,CAMPUS CAPANEMA,2
3,CAMPUS CAPANEMA/IFPR,89
4,CAMPUS COLOMBO/IFPR,69
5,CAMPUS JAGUARIAIVA/IFPR,65
6,CAMPUS MARANGUAPE,213
7,CAMPUS PECEM,92
8,CAMPUS PINHAIS,11
9,CAMPUS PINHAIS/IFPR,28


In [14]:
df_siafi_negativo['nm_ug_tabela_ug'].tolist()

['CAMPUS ACOPIARA',
 'CAMPUS AVANCADO DE ACOPIARA',
 'CAMPUS CAPANEMA',
 'CAMPUS CAPANEMA/IFPR',
 'CAMPUS COLOMBO/IFPR',
 'CAMPUS JAGUARIAIVA/IFPR',
 'CAMPUS MARANGUAPE',
 'CAMPUS PECEM',
 'CAMPUS PINHAIS',
 'CAMPUS PINHAIS/IFPR',
 'CAMPUS PITANGA/IFPR',
 'CAMPUS UNIAO DA VITORIA/IFPR',
 'DIRETORIA DE GESTAO INTERNA',
 'FUNDACAO UNIVERSIDADE FEDERAL',
 'FUNDACAO UNIVERSIDADE FEDERAL DE SAO CARLOS',
 'FUNDACAO UNIVERSIDADE FEDERAL DE SAO CARLOS - UFSCAR',
 'GESTAO E MANUTENCAO DO LEGADO OLIMPICO E PARAOLIMPICO',
 'IFNMG \x1a CAMPUS DIAMANTINA',
 'IFNMG - CAMPUS DIAMANTINA',
 'IFNMG CAMPUS DIAMANTINA',
 'IFPB CAMPUS SANTA LUZIA',
 'INST, FED, DE EDUC, CIENC, E TEC, DA PARAIBA - CAMPUS SANTA LUZIA',
 'INST. FED DE EDUC. CIENCIA E TECNOLOGIA DE MG- CAMPUS IBIRITE',
 'INST. FED. DE EDUC. CIENC. E TEC. DA PARAIBA - CAMPUS SANTA LUZIA',
 'INST.FED.DE EDUC.,CIENC.E TEC.DA PARAIBA \x1a SANTA LUZIA',
 'INSTITUTO FEDERAL DE ALAGOAS \x1a CAMPUS BATALHA',
 'INSTITUTO FEDERAL DE ALAGOAS \x1a CAMPUS 

In [15]:
lista_siafi = [26405,
 26405,
 26432,
 26432,
 26432,
 26432,
 26405,
 26405,
 26432,
 26432,
 26432,
 26432,
 55000,
 26279,
 26280,
 26280,
 51000,
 26410,
 26410,
 26410,
 26417,
 26417,
 26409,
 26417,
 26417,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26402,
 26421,
 26421,
 26439,
 26432,
 26432,
 26432,
 26432,
 26432,
 51000,
 51000,
 46000,
 46000,
 30000,
 30000,
 30000,
 26000,
 26000,
 30000,
 26455,
 26455]

In [16]:
df_siafi_negativo['cd_orgao_siafi_corrigido'] = lista_siafi

In [17]:
df_siafi_negativo

,nm_ug_tabela_ug,quantidade,cd_orgao_siafi_corrigido
0,CAMPUS ACOPIARA,73,26405
1,CAMPUS AVANCADO DE ACOPIARA,118,26405
2,CAMPUS CAPANEMA,2,26432
3,CAMPUS CAPANEMA/IFPR,89,26432
4,CAMPUS COLOMBO/IFPR,69,26432
5,CAMPUS JAGUARIAIVA/IFPR,65,26432
6,CAMPUS MARANGUAPE,213,26405
7,CAMPUS PECEM,92,26405
8,CAMPUS PINHAIS,11,26432
9,CAMPUS PINHAIS/IFPR,28,26432


In [18]:
# Criar um dicionário de mapeamento
mapa_siafi = dict(zip(df_siafi_negativo['nm_ug_tabela_ug'], df_siafi_negativo['cd_orgao_siafi_corrigido']))

# Aplicar a correção no df_total
df_total.loc[df_total['cd_orgao_siafi'] == -2, 'cd_orgao_siafi'] = df_total['nm_ug_tabela_ug'].map(mapa_siafi)

In [19]:
df_total[df_total['cd_orgao_siafi'] == -2]

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape


## Agrupar os contratos para validação dos dados de contrato

In [20]:
df_total.columns

Index(['id_terc', 'sg_orgao_sup_tabela_ug', 'cd_ug_gestora', 'nm_ug_tabela_ug',
       'sg_ug_gestora', 'nr_contrato', 'nr_cnpj', 'nm_razao_social', 'nr_cpf',
       'nm_terceirizado', 'nm_categoria_profissional', 'nm_escolaridade',
       'nr_jornada', 'nm_unidade_prestacao', 'vl_mensal_salario',
       'vl_mensal_custo', 'Num_Mes_Carga', 'Mes_Carga', 'Ano_Carga',
       'sg_orgao', 'nm_orgao', 'cd_orgao_siafi', 'cd_orgao_siape'],
      dtype='object')

In [21]:
# Selecionar apenas as colunas necessárias
colunas_necessarias = [
    'sg_orgao_sup_tabela_ug', 'cd_ug_gestora', 'nm_ug_tabela_ug',
    'sg_ug_gestora', 'nr_contrato', 'nr_cnpj', 'nm_razao_social', 
    'nm_unidade_prestacao', 'sg_orgao', 'nm_orgao', 
    'cd_orgao_siafi', 'cd_orgao_siape'
]

df_reduzido = df_total[colunas_necessarias].copy()

In [22]:
# Converter colunas com potencial problema para string usando .loc[]
colunas_texto = ['nm_unidade_prestacao', 'sg_orgao', 'nm_orgao']
for coluna in colunas_texto:
    df_reduzido.loc[:, coluna] = df_reduzido[coluna].fillna('').astype(str)

In [23]:
# Agrupar por 'nr_contrato', 'cd_orgao_siafi' e 'nm_razao_social'
df_consolidado_validacao = df_reduzido.groupby(['nr_contrato', 'cd_orgao_siafi', 'nm_razao_social']).agg({
    'sg_orgao_sup_tabela_ug': 'first',
    'cd_ug_gestora': 'first',
    'nm_ug_tabela_ug': 'first',
    'sg_ug_gestora': 'first',
    'nr_cnpj': 'first',
    'nm_unidade_prestacao': lambda x: ', '.join(x.unique()),  
    'sg_orgao': lambda x: ', '.join(x.unique()),
    'nm_orgao': lambda x: ', '.join(x.unique()),
    'cd_orgao_siape': 'first'  
}).reset_index()

In [24]:
df_consolidado_validacao.head()

,nr_contrato,cd_orgao_siafi,nm_razao_social,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_cnpj,nm_unidade_prestacao,sg_orgao,nm_orgao,cd_orgao_siape
0,000002019,26273,SULCLEAN SERVICOS LTDA.,MIN.EDUC,154042,UNIVERIDADE FEDERAL DO RIO GRANDE - FURG,FURG,6205427000102,CIDEC / REITORIA,FURG,UNIVERSIDADE FEDERAL DO RIO GRANDE - FURG,26273
1,000002023,20411,XYZ LTDA,MINC,343029,SUPERINTENDENCIA DO IPHAN NO DISTRITO FEDERAL,IPHAN-DF,0,IPHAN - DF,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,24204
2,000012011,20411,POLO SEGURANCA ESPECIALIZADA LTDA.,MINIST.,343002,SUPERINTENDENCIA DO IPHAN NO PARA,IPHAN-PA,02650833000123,PREDIO SEDE/PA,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,24204
3,000012012,20411,ARE SERVICOS LTDA,MINIST.,343003,IPHAN - SUPERINTENDENCIA DO MARANHAO,IPHAN-MA,11873594000161,SUPERINTENDENCIA,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,24204
4,000012012,36212,RM SERVILOS EMPRESARIAIS LTDA-ME,MINISTER,253029,COORDENACAO ESTADUAL DE VIGILANCIA SANITARIA D...,CVSPAF-AC/ANVISA,09646758000176,CVPAF/AC-SEDE,ANVS,AGENCIA NACIONAL DE VIGILANCIA SANITARIA,36207


In [25]:
df_consolidado_validacao.shape

(22055, 12)

## Padronizar o CPNJ com o formato da API

In [26]:
def formatar_cnpj(cnpj):
    """Formata um CNPJ no padrão 'XX.XXX.XXX/XXXX-XX'"""
    cnpj = re.sub(r'\D', '', str(cnpj))  # Remove tudo que não é número
    cnpj = cnpj.zfill(14)  # Completa com zeros à esquerda se tiver menos de 14 dígitos
    if len(cnpj) == 14:
        return f"{cnpj[:2]}.{cnpj[2:5]}.{cnpj[5:8]}/{cnpj[8:12]}-{cnpj[12:]}"
    return ""  # Retorna vazio se não for um CNPJ válido

In [27]:
# Criar a nova coluna 'cnpj_formatado'
df_consolidado_validacao['cnpj_formatado'] = df_consolidado_validacao['nr_cnpj'].apply(formatar_cnpj)

In [28]:
df_consolidado_validacao.head()

,nr_contrato,cd_orgao_siafi,nm_razao_social,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_cnpj,nm_unidade_prestacao,sg_orgao,nm_orgao,cd_orgao_siape,cnpj_formatado
0,000002019,26273,SULCLEAN SERVICOS LTDA.,MIN.EDUC,154042,UNIVERIDADE FEDERAL DO RIO GRANDE - FURG,FURG,6205427000102,CIDEC / REITORIA,FURG,UNIVERSIDADE FEDERAL DO RIO GRANDE - FURG,26273,06.205.427/0001-02
1,000002023,20411,XYZ LTDA,MINC,343029,SUPERINTENDENCIA DO IPHAN NO DISTRITO FEDERAL,IPHAN-DF,0,IPHAN - DF,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,24204,00.000.000/0000-00
2,000012011,20411,POLO SEGURANCA ESPECIALIZADA LTDA.,MINIST.,343002,SUPERINTENDENCIA DO IPHAN NO PARA,IPHAN-PA,02650833000123,PREDIO SEDE/PA,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,24204,02.650.833/0001-23
3,000012012,20411,ARE SERVICOS LTDA,MINIST.,343003,IPHAN - SUPERINTENDENCIA DO MARANHAO,IPHAN-MA,11873594000161,SUPERINTENDENCIA,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,24204,11.873.594/0001-61
4,000012012,36212,RM SERVILOS EMPRESARIAIS LTDA-ME,MINISTER,253029,COORDENACAO ESTADUAL DE VIGILANCIA SANITARIA D...,CVSPAF-AC/ANVISA,09646758000176,CVPAF/AC-SEDE,ANVS,AGENCIA NACIONAL DE VIGILANCIA SANITARIA,36207,09.646.758/0001-76


In [29]:
#df_consolidado_validacao.to_csv("../data/terceirizados/arquivos_tratados/consolidado_validacao.csv", index=False)